In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [2]:
cfips = pd.read_csv("./godaddy-microbusiness-density-forecasting/census_starter.csv")
cfips.head()

,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,cfips,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,...,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
0,76.6,78.9,80.6,82.7,85.5,1001,14.5,15.9,16.1,16.7,...,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1,74.5,78.1,81.8,85.1,87.9,1003,20.4,20.7,21.0,20.2,...,1.4,1.3,1.4,1.0,1.3,52562,55962.0,58320,61756.0,64346.0
2,57.2,60.4,60.5,64.6,64.6,1005,7.6,7.8,7.6,7.3,...,0.5,0.3,0.8,1.1,0.8,33368,34186.0,32525,34990.0,36422.0
3,62.0,66.1,69.2,76.1,74.6,1007,8.1,7.6,6.5,7.4,...,1.2,1.4,1.6,1.7,2.1,43404,45340.0,47542,51721.0,54277.0
4,65.8,68.5,73.0,79.6,81.0,1009,8.7,8.1,8.6,8.9,...,1.3,1.4,0.9,1.1,0.9,47412,48695.0,49358,48922.0,52830.0


In [3]:
testing_data = pd.read_csv("./godaddy-microbusiness-density-forecasting/test.csv")
testing_data = pd.merge(testing_data, cfips, how="left", on="cfips")

testing_data["first_day_of_month"] = \
    pd.to_datetime(testing_data["first_day_of_month"])

testing_data["month"] = testing_data["first_day_of_month"].dt.month

testing_data.to_csv("./godaddy-microbusiness-density-forecasting/test_GoDaddy.csv", index=False)

testing_data.head()


,row_id,cfips,first_day_of_month,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,pct_college_2017,pct_college_2018,...,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021,month
0,1001_2022-11-01,1001,2022-11-01,76.6,78.9,80.6,82.7,85.5,14.5,15.9,...,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0,11
1,1003_2022-11-01,1003,2022-11-01,74.5,78.1,81.8,85.1,87.9,20.4,20.7,...,1.3,1.4,1.0,1.3,52562,55962.0,58320,61756.0,64346.0,11
2,1005_2022-11-01,1005,2022-11-01,57.2,60.4,60.5,64.6,64.6,7.6,7.8,...,0.3,0.8,1.1,0.8,33368,34186.0,32525,34990.0,36422.0,11
3,1007_2022-11-01,1007,2022-11-01,62.0,66.1,69.2,76.1,74.6,8.1,7.6,...,1.4,1.6,1.7,2.1,43404,45340.0,47542,51721.0,54277.0,11
4,1009_2022-11-01,1009,2022-11-01,65.8,68.5,73.0,79.6,81.0,8.7,8.1,...,1.4,0.9,1.1,0.9,47412,48695.0,49358,48922.0,52830.0,11


In [4]:
training_data = pd.read_csv("./godaddy-microbusiness-density-forecasting/train.csv")[["row_id", "cfips", "first_day_of_month", "microbusiness_density"]]
training_data = pd.merge(training_data, cfips, how="left", on="cfips")

training_data["first_day_of_month"] = \
    pd.to_datetime(training_data["first_day_of_month"])

training_data["month"] = training_data["first_day_of_month"].dt.month

training_data.to_csv("./godaddy-microbusiness-density-forecasting/train_GoDaddy.csv", index=False)

training_data.head()


,row_id,cfips,first_day_of_month,microbusiness_density,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,pct_college_2017,...,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021,month
0,1001_2019-08-01,1001,2019-08-01,3.007682,76.6,78.9,80.6,82.7,85.5,14.5,...,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0,8
1,1001_2019-09-01,1001,2019-09-01,2.884870,76.6,78.9,80.6,82.7,85.5,14.5,...,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0,9
2,1001_2019-10-01,1001,2019-10-01,3.055843,76.6,78.9,80.6,82.7,85.5,14.5,...,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0,10
3,1001_2019-11-01,1001,2019-11-01,2.993233,76.6,78.9,80.6,82.7,85.5,14.5,...,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0,11
4,1001_2019-12-01,1001,2019-12-01,2.993233,76.6,78.9,80.6,82.7,85.5,14.5,...,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0,12


In [5]:
print(training_data.info())
print(testing_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122265 entries, 0 to 122264
Data columns (total 30 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   row_id                 122265 non-null  object        
 1   cfips                  122265 non-null  int64         
 2   first_day_of_month     122265 non-null  datetime64[ns]
 3   microbusiness_density  122265 non-null  float64       
 4   pct_bb_2017            122265 non-null  float64       
 5   pct_bb_2018            122265 non-null  float64       
 6   pct_bb_2019            122265 non-null  float64       
 7   pct_bb_2020            122265 non-null  float64       
 8   pct_bb_2021            122265 non-null  float64       
 9   pct_college_2017       122265 non-null  float64       
 10  pct_college_2018       122265 non-null  float64       
 11  pct_college_2019       122265 non-null  float64       
 12  pct_college_2020       122265 non-null  floa

In [6]:
X = training_data.drop(columns={"row_id", "first_day_of_month", "microbusiness_density",
 "pct_bb_2017", "pct_bb_2018", "pct_bb_2019",
 "pct_foreign_born_2017", "pct_foreign_born_2018", "pct_foreign_born_2019",
 "pct_it_workers_2017", "pct_it_workers_2018", "pct_it_workers_2019",
 "median_hh_inc_2017", "median_hh_inc_2018", "median_hh_inc_2019"
    })
y = training_data["microbusiness_density"]

X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_pred = testing_data.drop(columns={"row_id", "first_day_of_month",
 "pct_bb_2017", "pct_bb_2018", "pct_bb_2019",
 "pct_foreign_born_2017", "pct_foreign_born_2018", "pct_foreign_born_2019",
 "pct_it_workers_2017", "pct_it_workers_2018", "pct_it_workers_2019",
 "median_hh_inc_2017", "median_hh_inc_2018", "median_hh_inc_2019"
    })

X_pred.head()

,cfips,pct_bb_2020,pct_bb_2021,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,pct_college_2021,pct_foreign_born_2020,pct_foreign_born_2021,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2020,median_hh_inc_2021,month
0,1001,82.7,85.5,14.5,15.9,16.1,16.7,16.4,2.3,2.1,0.6,1.1,57982.0,62660.0,11
1,1003,85.1,87.9,20.4,20.7,21.0,20.2,20.6,3.4,3.5,1.0,1.3,61756.0,64346.0,11
2,1005,64.6,64.6,7.6,7.8,7.6,7.3,6.7,2.6,2.6,1.1,0.8,34990.0,36422.0,11
3,1007,76.1,74.6,8.1,7.6,6.5,7.4,7.9,1.6,1.1,1.7,2.1,51721.0,54277.0,11
4,1009,79.6,81.0,8.7,8.1,8.6,8.9,9.3,4.4,4.5,1.1,0.9,48922.0,52830.0,11


In [7]:
import numpy as np

def SMAPE(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [8]:
from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=10000, eval_metric=SMAPE)
model.fit(X_train, y_train, 
             early_stopping_rounds=10, 
             eval_set=[(X_test, y_test)],
             
             
             verbose=False)

y_pred = model.predict(X_pred)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [9]:
testing_data["microbusiness_density"] = pd.Series(y_pred).astype(float)
testing_data[["row_id", "microbusiness_density"]].to_csv("./godaddy-microbusiness-density-forecasting/submission_XGBOOST.csv", index=False)

In [10]:
df = pd.read_csv('./godaddy-microbusiness-density-forecasting/submission_XGBOOST.csv')
df.head()

,row_id,microbusiness_density
0,1001_2022-11-01,3.138485
1,1003_2022-11-01,7.850580
2,1005_2022-11-01,1.092642
3,1007_2022-11-01,1.215565
4,1009_2022-11-01,1.657145
